In [1]:
import pandas as pd

In [4]:
images_df = pd.read_csv('data/wit_v1.train.all-00000-of-00010.tsv.gz', compression='gzip', sep='\t', encoding='iso-8859-1')
print(images_df.shape)
images_df.head()

(3708026, 17)


,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
0,en,https://en.wikipedia.org/wiki/Oxydactylus,https://upload.wikimedia.org/wikipedia/commons...,Oxydactylus,NaN,Oxydactylus,NaN,English: Mounted skeleton of Oxydactylus longi...,NaN,image/jpeg,3564,2748,True,True,False,Oxydactylus is an extinct genus of camelid end...,Oxydactylus is an extinct genus of camelid end...
1,cs,https://cs.wikipedia.org/wiki/Mechanick%C3%A1_...,http://upload.wikimedia.org/wikipedia/commons/...,MechanickÃ¡ mÃ­Åidla,NaN,MechanickÃ¡ mÃ­Åidla,MechanickÃ¡ mÃ­Åidla na ÃºtoÄnÃ© puÅ¡ce M16,M16 rifle correct sight picture,NaN,image/png,606,1182,False,False,False,MechanickÃ¡ mÃ­Åidla je zaÅÃ­zenÃ­ urÄenÃ© ...,MechanickÃ¡ mÃ­Åidla je zaÅÃ­zenÃ­ urÄenÃ© ...
2,sq,https://sq.wikipedia.org/wiki/Mjedisi_natyror,https://upload.wikimedia.org/wikipedia/commons...,Mjedisi natyror,NaN,Mjedisi natyror,Menaxhimi i ujÃ«rave dhe tokÃ«s ka ruajtur kar...,"English: Hopetoun Falls, Beech Forest, near Ot...",NaN,image/jpeg,2048,3072,False,False,True,"Mjedis natyror quhet vendi, njerÃ«zit, gjÃ«rat...","Mjedis natyror quhet vendi, njerÃ«zit, gjÃ«rat..."
3,nl,https://nl.wikipedia.org/wiki/Zeesterren,http://upload.wikimedia.org/wikipedia/commons/...,Zeesterren,Armen,Zeesterren / Uiterlijke kenmerken / Armen,Voorzijde van een arm met buisvoetjes aan de o...,FranÃ§ais&#160;: Marthasteria glacialis (Linna...,NaN,image/jpeg,2112,3200,False,True,False,Zeesterren zijn een groep van ongewervelde die...,De armen van de zeester moeten niet gezien wor...
4,de,https://de.wikipedia.org/wiki/Friedrich_von_%C...,https://upload.wikimedia.org/wikipedia/commons...,Friedrich von Ãsterreich-Teschen,Erster Weltkrieg,Friedrich von Ãsterreich-Teschen / Erster Wel...,Denkmal Erzherzog Friedrichs in MosonmagyarÃ³v...,"English: Monument to Frederic Habsburg, Mosonm...",NaN,image/jpeg,2043,1570,False,False,False,Erzherzog Friedrich Maria Albrecht Wilhelm Kar...,Friedrich sollte 1914 wegen seiner Disharmonie...


In [17]:
# Notice that some images are present in the images_df dataset multiple times in the same language. This means that the image is part 
# of different articles at the same time. Which article's labels should we assign to the image then? We use the dumb heuristic to 
# assign the first article's labels to the image and ignore the rest as it is not final goal of the project.
print(f'The images are unique i.e. belong to only one article: {images_df.image_url.is_unique} \n')
print(images_df.image_url.value_counts())
images_df[images_df.image_url == 'https://upload.wikimedia.org/wikipedia/commons/d/db/Moonmap_from_clementine_data.png']

The images are unique i.e. belong to only one article: False 

https://upload.wikimedia.org/wikipedia/commons/c/c7/North_Macedonia_relief_location_map.jpg             286
https://upload.wikimedia.org/wikipedia/commons/1/1a/E3d_txikia.png                                      157
https://upload.wikimedia.org/wikipedia/commons/d/db/Moonmap_from_clementine_data.png                    126
http://upload.wikimedia.org/wikipedia/commons/b/bb/Location_map_South_Georgia.png                        95
https://upload.wikimedia.org/wikipedia/commons/f/ff/Espa%C3%B1aLoc.svg                                   80
                                                                                                       ... 
https://upload.wikimedia.org/wikipedia/commons/7/78/Keizo_Obuchi_cropped_1_Keizo_Obuchi_19980730.jpg      1
https://upload.wikimedia.org/wikipedia/commons/a/ac/Triumph_GT6_Brick_Lane_1.jpg                          1
https://upload.wikimedia.org/wikipedia/commons/c/c2/Makahiku_Falls.jpg   

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
16117,fa,https://fa.wikipedia.org/wiki/%D8%A8%D9%86%DA%...,https://upload.wikimedia.org/wikipedia/commons...,بنکرفت (دهانه),NaN,بنکرفت (دهانه),موقعیت بنکرفت (دهانه) در ماه,English: cylindrical map projection of the Moo...,بنکرفت (دهانه) در ماه قرار گرفته‌است,image/png,720,1440,False,False,False,بنکرفت (دهانه) یک دهانه برخوردی در ماه‌ است.,بنکرفت (دهانه) یک دهانه برخوردی در ماه‌ است.
23894,fa,https://fa.wikipedia.org/wiki/%D8%B1%DB%8C%D8%...,https://upload.wikimedia.org/wikipedia/commons...,ریتنهوس (دهانه),NaN,ریتنهوس (دهانه),موقعیت ریتنهوس (دهانه) در ماه,English: cylindrical map projection of the Moo...,ریتنهوس (دهانه) در ماه واقع شده‌است,image/png,720,1440,False,False,False,ریتنهوس (دهانه) یک دهانه برخوردی در ماه‌ است.,ریتنهوس (دهانه) یک دهانه برخوردی در ماه‌ است.
39260,fa,https://fa.wikipedia.org/wiki/%D9%84%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,لیندبلد (دهانه),NaN,لیندبلد (دهانه),موقعیت لیندبلد (دهانه) در ماه,English: cylindrical map projection of the Moo...,لیندبلد (دهانه) در ماه قرار گرفته‌است,image/png,720,1440,False,False,False,لیندبلد (دهانه) یک دهانه برخوردی در ماه‌ است.,لیندبلد (دهانه) یک دهانه برخوردی در ماه‌ است.
55856,fa,https://fa.wikipedia.org/wiki/%D9%84%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,لیندسی (دهانه),NaN,لیندسی (دهانه),موقعیت لیندسی (دهانه) در ماه,English: cylindrical map projection of the Moo...,لیندسی (دهانه) در ماه واقع شده‌است,image/png,720,1440,False,False,False,لیندسی (دهانه) یک دهانه برخوردی در ماه‌ است.,لیندسی (دهانه) یک دهانه برخوردی در ماه‌ است.
97734,fa,https://fa.wikipedia.org/wiki/%D9%88%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,وینلاک (دهانه),NaN,وینلاک (دهانه),موقعیت وینلاک (دهانه) در ماه,English: cylindrical map projection of the Moo...,وینلاک (دهانه) در ماه قرار گرفته‌است,image/png,720,1440,False,False,False,وینلاک (دهانه) یک دهانه برخوردی در ماه‌ است.,وینلاک (دهانه) یک دهانه برخوردی در ماه‌ است.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3626080,fa,https://fa.wikipedia.org/wiki/%D9%88%D8%A7%D8%...,https://upload.wikimedia.org/wikipedia/commons...,واتس (دهانه),NaN,واتس (دهانه),موقعیت واتس (دهانه) در ماه,English: cylindrical map projection of the Moo...,واتس (دهانه) در ماه واقع شده‌است,image/png,720,1440,False,False,False,واتس (دهانه) یک دهانه برخوردی در ماه‌ است.,واتس (دهانه) یک دهانه برخوردی در ماه‌ است.
3639871,fa,https://fa.wikipedia.org/wiki/%D9%88%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,ویپل (دهانه),NaN,ویپل (دهانه),موقعیت ویپل (دهانه) در ماه,English: cylindrical map projection of the Moo...,ویپل (دهانه) در ماه واقع شده‌است,image/png,720,1440,False,False,False,ویپل (دهانه) یک دهانه برخوردی در ماه‌ است.,ویپل (دهانه) یک دهانه برخوردی در ماه‌ است.
3643212,fa,https://fa.wikipedia.org/wiki/%D9%88%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,ویلهلم (دهانه),NaN,ویلهلم (دهانه),موقعیت ویلهلم (دهانه) در ماه,English: cylindrical map projection of the Moo...,ویلهلم (دهانه) در ماه قرار گرفته‌است,image/png,720,1440,False,False,False,ویلهلم (دهانه) یک دهانه برخوردی در ماه‌ است.,ویلهلم (دهانه) یک دهانه برخوردی در ماه‌ است.
3665841,fa,https://fa.wikipedia.org/wiki/%DB%8C%D9%88%D8%...,https://upload.wikimedia.org/wikipedia/commons...,یوری (دهانه),NaN,یوری (دهانه),موقعیت یوری (دهانه) در ماه,English: cylindrical map projection of the Moo...,یوری (دهانه) در ماه واقع شده‌است,image/png,720,1440,False,False,False,یوری (دهانه) یک دهانه برخوردی در ماه‌ است.,یوری (دهانه) یک دهانه برخوردی در ماه‌ است.


In [18]:
# Only keep unique values of image_url in images_df
unique_images = images_df.drop_duplicates(subset=['image_url'])
print(f'The images are unique i.e. belong to only one article: {unique_images.image_url.is_unique}')
print(unique_images.shape)
unique_images.head()

The images are unique i.e. belong to only one article: True
(2623494, 17)


,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
0,en,https://en.wikipedia.org/wiki/Oxydactylus,https://upload.wikimedia.org/wikipedia/commons...,Oxydactylus,NaN,Oxydactylus,NaN,English: Mounted skeleton of Oxydactylus longi...,NaN,image/jpeg,3564,2748,True,True,False,Oxydactylus is an extinct genus of camelid end...,Oxydactylus is an extinct genus of camelid end...
1,cs,https://cs.wikipedia.org/wiki/Mechanick%C3%A1_...,http://upload.wikimedia.org/wikipedia/commons/...,Mechanická mířidla,NaN,Mechanická mířidla,Mechanická mířidla na útočné pušce M16,M16 rifle correct sight picture,NaN,image/png,606,1182,False,False,False,Mechanická mířidla je zařízení určené pro zamí...,Mechanická mířidla je zařízení určené pro zamí...
2,sq,https://sq.wikipedia.org/wiki/Mjedisi_natyror,https://upload.wikimedia.org/wikipedia/commons...,Mjedisi natyror,NaN,Mjedisi natyror,Menaxhimi i ujërave dhe tokës ka ruajtur karak...,"English: Hopetoun Falls, Beech Forest, near Ot...",NaN,image/jpeg,2048,3072,False,False,True,"Mjedis natyror quhet vendi, njerëzit, gjërat, ...","Mjedis natyror quhet vendi, njerëzit, gjërat, ..."
3,nl,https://nl.wikipedia.org/wiki/Zeesterren,http://upload.wikimedia.org/wikipedia/commons/...,Zeesterren,Armen,Zeesterren / Uiterlijke kenmerken / Armen,Voorzijde van een arm met buisvoetjes aan de o...,Français&#160;: Marthasteria glacialis (Linnae...,NaN,image/jpeg,2112,3200,False,True,False,Zeesterren zijn een groep van ongewervelde die...,De armen van de zeester moeten niet gezien wor...
4,de,https://de.wikipedia.org/wiki/Friedrich_von_%C...,https://upload.wikimedia.org/wikipedia/commons...,Friedrich von Österreich-Teschen,Erster Weltkrieg,Friedrich von Österreich-Teschen / Erster Welt...,Denkmal Erzherzog Friedrichs in Mosonmagyaróva...,"English: Monument to Frederic Habsburg, Mosonm...",NaN,image/jpeg,2043,1570,False,False,False,Erzherzog Friedrich Maria Albrecht Wilhelm Kar...,Friedrich sollte 1914 wegen seiner Disharmonie...


In [19]:
articles_df = pd.read_json('enwiki.labeled_article_items.json.bz2', compression='bz2', lines=True)
print(articles_df.shape)
articles_df

(5926244, 9)


,article_pid,wp_templates,article_revid,qid,sitelinks,title,talk_revid,talk_pid,taxo_labels
0,18951386.0,"[WikiProject Objectivism, WikiProject Novels, ...",926765055.0,Q374098,"{'la': 'Atlas Shrugged', 'sv': 'Och världen sk...",Atlas Shrugged,911346471,128,"[Culture.Media.Books, History and Society.Poli..."
1,358.0,"[WikiProject Africa, WikiProject Countries, Wi...",928541225.0,Q262,"{'crh': 'Cezair', 'ku': 'Cezayir', 'et': 'Alže...",Algeria,927128572,354,"[Geography.Geographical, History and Society.S..."
2,2482.0,"[WikiProject France, WikiProject Architecture,...",924947879.0,Q64436,"{'sv': 'Triumfbågen, Paris', 'en': 'Arc de Tri...",Arc de Triomphe,921129575,672,"[Culture.Visual arts.Visual arts*, Geography.G..."
3,18951655.0,"[WikiProject Archaeology, WikiProject Anthropo...",926027145.0,Q23498,"{'ku': 'Arkeolojî', 'tg': 'Бостоншиносӣ', 'ro'...",Archaeology,896487747,692,"[History and Society.History, History and Soci..."
4,713.0,"[WikiProject Robotics, WikiProject Science Fic...",917854604.0,Q181787,"{'sv': 'Android', 'en': 'Android (robot)', 'ja...",Android (robot),899920932,714,"[Culture.Media.Entertainment, STEM.Technology,..."
...,...,...,...,...,...,...,...,...,...
5926239,62324504.0,[WikiProject Professional wrestling],928858375.0,None,[],La Copa Junior (2012),928855715,62480245,"[Culture.Media.Entertainment, Culture.Media.Te..."
5926240,62324502.0,[WikiProject Professional wrestling],928858654.0,None,[],La Copa Junior (2010),928855740,62480246,"[Culture.Media.Entertainment, Culture.Media.Te..."
5926241,62324499.0,[WikiProject Professional wrestling],928854421.0,None,[],La Copa Junior (2006),928855761,62480247,"[Culture.Media.Entertainment, Culture.Media.Te..."
5926242,62324495.0,[WikiProject Professional wrestling],928854231.0,None,[],La Copa Junior (2005),928855787,62480248,"[Culture.Media.Entertainment, Culture.Media.Te..."


In [20]:
article_labels = unique_images.merge(articles_df, left_on=['page_title'], right_on=['title'])
print(article_labels.shape)

(898358, 26)


In [21]:
articles_and_labels = article_labels[['image_url', 'page_url', 'page_title', 'taxo_labels']]
articles_and_labels

,image_url,page_url,page_title,taxo_labels
0,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/Oxydactylus,Oxydactylus,"[STEM.Biology, STEM.STEM*, STEM.Earth and envi..."
1,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/Maine%27s_3rd_co...,Maine's 3rd congressional district,"[History and Society.Politics and government, ..."
2,https://upload.wikimedia.org/wikipedia/commons...,https://pt.wikipedia.org/wiki/Akseli_Gallen-Ka...,Akseli Gallen-Kallela,"[Culture.Biography.Biography*, Geography.Regio..."
3,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/Akseli_Gallen-Ka...,Akseli Gallen-Kallela,"[Culture.Biography.Biography*, Geography.Regio..."
4,https://upload.wikimedia.org/wikipedia/commons...,https://da.wikipedia.org/wiki/Akseli_Gallen-Ka...,Akseli Gallen-Kallela,"[Culture.Biography.Biography*, Geography.Regio..."
...,...,...,...,...
898353,https://upload.wikimedia.org/wikipedia/commons...,https://ru.wikipedia.org/wiki/Canon_EOS_5D_Mar...,Canon EOS 5D Mark III,"[Culture.Visual arts.Visual arts*, History and..."
898354,https://upload.wikimedia.org/wikipedia/commons...,https://de.wikipedia.org/wiki/Bure%C3%A5,Bureå,"[Geography.Regions.Europe.Northern Europe, Geo..."
898355,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/Lawrence_G._McDo...,Lawrence G. McDonald,[Culture.Biography.Biography*]
898356,http://upload.wikimedia.org/wikipedia/commons/...,https://en.wikipedia.org/wiki/Battenberg_Castle,Battenberg Castle,"[Culture.Visual arts.Visual arts*, Culture.Vis..."


In [22]:
articles_and_labels.to_json('articles_and_labels.json.bz2', compression='bz2')